In [ ]:
# This code here is intended to create a custom keras layer.
# This layer takes as input the path to the image
# It outputs an array that can be given as an input to a convolutional neural network

In [12]:
#Create constants and import libraries
UNPROCESSED_DIRECTORY = "CNN_synth_testset"
PROCESSED_DIRECTORY = "preprocessedImages"
X_DIM = 256
Y_DIM = 256
import os
import numpy as np
from PIL import Image as imLib
from keras.layers import Layer
from keras import models
from keras import layers
from keras.utils import Sequence
from math import ceil


In [8]:
#Build custom layer

class ImageBatchSequence(Sequence):
    def __init__(self, x_set, batch_size):
        self.x = x_set
        y_set = [(1 if "FAKE" in x else 0) for x in x_set] #This makes the output to be 1 (AI-generated) if FAKE is in pathname.
        #                                                  # Otherwise, it makes the output 0 (not AI-generated)
        self.y = y_set
        self.batch_size = batch_size
    def __len__(self):
        return ceil(len(self.x)/self.batch_size)
    def __getitem__(self, index):
        low = index * self.batch_size
        high = min(low+self.batch_size, len(self.x))
        batch_x = self.x[low : high]
        batch_y = self.y[low : high]
        image_x_batch = []
        for imPath in batch_x:
            image = np.asarray(imLib.open(imPath))
            image_x_batch.append(image)
        return np.array(image_x_batch), np.array(batch_y)


In [13]:
model = models.Sequential()
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
tempList=['./CNN_synth_testset\\biggan\\0_real\\0--n01440764_5043.png', './CNN_synth_testset\\biggan\\0_real\\0--n01440764_8600.png', './CNN_synth_testset\\biggan\\0_real\\10--n01530575_10395.png', './CNN_synth_testset\\biggan\\0_real\\100--n01860187_3168.png', './CNN_synth_testset\\biggan\\0_real\\101--n01871265_1128.png']
ImageBatchGenerator = ImageBatchSequence(x_set = tempList, batch_size = 32)


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________
